# Verifiaction of acoustic modeling with the MMS method

We verify the accuracy and convergence of our finite-difference acoustic modeling kernel in an heterogeneous belocity model with the MMS method

# Seismic modelling with devito

We describe here a step by step setup of seismic modelling with Devito in a simple 2D case. We will create a physical model of our domain and define a single source and an according set of receivers to model for the forward model. But first, we initialize some basic utilities.

In [1]:
from devito import *

In [2]:
import numpy as np
%matplotlib inline

In [3]:
from examples.seismic import Model
# Model with fixed time step value
class ModelBench(Model):
    """
    Physical model used for accuracy benchmarking.
    The critical dt is made small enough to ignore
    time discretization errors
    """

    @property
    def critical_dt(self):
        """Critical computational time step value."""
        return .1*self.spacing[0]

In [4]:
# Discretization order
orders = (2, 4, 6, 8, 10)
norder = len(orders)

# Domain sizes and gird spacing
shapes = ((201, 2.0), (161, 2.5), (101, 4.0))
dx = [2.0, 2.5, 4.0]
nshapes = len(shapes)

# Number of time steps
nt = 1501
# Time axis
time = np.linspace(0., 150., nt)
# Source peak frequency
f0 = .07

# Make a three layers velocity model

In [5]:
v = 1.5 * np.ones((201, 201))
v[:, 75:] = 2.5
v[:, 125:] = 3.0
model = ModelBench(vp=v, origin=(0., 0.), spacing=(.005, .005), shape=(201, 201), nbpml=0, space_order=0, dtype=np.float64)

# Analytical solution definition

In [6]:
x, z = model.grid.dimensions
t = model.grid.time_dim

In [7]:
from sympy import cos, sin, exp, pprint, sqrt, sin, sinc

In [8]:
# sol = sin(2*np.pi*t.spacing*t/10)* exp(-(x.spacing*x - 100)**2/200 - (z.spacing*z - 100)**2/200)
import sympy
u, v, w, c = sympy.symbols('u v w c')
sol = sin(2*np.pi*u) * sin(4*np.pi*v)* sin(2*np.pi*w)

# Corresponding source

In [9]:
source =  model.m*sol.diff(w, w) - sol.diff(u, u) - sol.diff(v, v)

In [10]:
sol = sol.subs({u: x*x.spacing, v: z*z.spacing, w: t*t.spacing, c: sqrt(1/model.m)})
source = source.subs({u: x*x.spacing, v: z*z.spacing, w: t*t.spacing, c: sqrt(1/model.m)})

# Wave equation

In [11]:
u = TimeFunction(name="u", grid=model.grid, space_order=16, time_order=2)
u_true = TimeFunction(name="utrue", grid=model.grid, space_order=0, time_order=2)
src = TimeFunction(name="src", grid=model.grid, space_order=0, time_order=2)

In [12]:
wave_eq = Eq(u.forward, 2 * u - u.backward + t.spacing**2/model.m * (u.laplace + src))

In [13]:
true_eqq = Eq(u_true, sol)

In [14]:
src_eq = Eq(src, source)

In [15]:
# inital conditions, t=1
u_1 = Eq(u.subs(t, 1), sol.subs(t, 1))

In [16]:
## from devito import configuration
configuration["log_level"] = "ERROR"
h = .00125
op = Operator([u_1, true_eqq, src_eq, wave_eq], subs=({t.spacing: .1*h, x.spacing:h, z.spacing:h}))
op.apply(t_m=1, t_M=100)

PerformanceSummary([('section0',
                     PerfEntry(time=0.06825099999999996, gflopss=3.788463172700768, gpointss=0.236778948293798, oi=1.0843484086238993, ops=64, itershapes=[(100, 201, 201), (100, 201, 201)]))])

In [17]:
def mms_sol(model):
    u, v, w, c = sympy.symbols('u v w c')
    sol = sin(2*np.pi*u) * sin(4*np.pi*v)* sin(2*np.pi*w)
    source =  model.m*sol.diff(w, w) - sol.diff(u, u) - sol.diff(v, v)
    x, z = model.grid.dimensions
    t = model.grid.time_dim
    
    sol = sol.subs({u: x*x.spacing, v: z*z.spacing, w: t*t.spacing, c: sqrt(1/model.m)})
    source = source.subs({u: x*x.spacing, v: z*z.spacing, w: t*t.spacing, c: sqrt(1/model.m)})
    
    return sol, source

In [91]:
def get_solution(size):
    clear_cache()
    # Model
    v = 1.5 * np.ones((size, size))
    v[:, 75:] = 2.5
    v[:, 125:] = 3.0
    model = ModelBench(vp=v, origin=(0., 0.), spacing=(.005, .005), shape=(size, size),
                       nbpml=0, space_order=0, dtype=np.float64)
    x, z = model.grid.dimensions
    t = model.grid.time_dim
    # Grid
    h = 1./(size - 1)
    dt = .1*h
    nt = int(1/dt)
    # Wave equation
    u = TimeFunction(name="u", grid=model.grid, space_order=2, time_order=2, save=nt)
    u_true = TimeFunction(name="utrue", grid=model.grid, space_order=2, time_order=0, save=nt)
    src = TimeFunction(name="src", grid=model.grid, space_order=2, time_order=0, save=nt)

    sol, source = mms_sol(model)
    
    wave_eq = Eq(u.forward, 2 * u - u.backward + t.spacing**2/model.m * (u.laplace + src))
    true_eqq = Eq(u_true, sol)

    src_eq = Eq(src, source)
    
    u_1 = Eq(u.subs(t, 1), sol.subs(t, 1))

    op = Operator([u_1, true_eqq, src_eq, wave_eq], subs=({t.spacing: .1*h, x.spacing:h, z.spacing:h}))
    op.apply()

    error = np.linalg.norm(u.data[(nt-3), 2:-2, 2:-2].reshape(-1) - u_true.data[(nt-3), 2:-2, 2:-2].reshape(-1), np.Inf)*h

    print("for h = %f and dt= %f error is %f" % (h, dt, error))

# MMS

In [92]:
size = [26, 51, 101]
t = 2

for s in size:
    get_solution(s)

for h = 0.040000 and dt= 0.004000 error is 0.002177
for h = 0.020000 and dt= 0.002000 error is 0.000757
for h = 0.010000 and dt= 0.001000 error is 0.000311


In [86]:
0.001312/0.000263

4.988593155893536